# Aplicación de Machine Learning para clasificar personas con cardiopatías

## Abstract

En este proyecto, se analizaron un conjunto de datos que contienen información relevante sobre factores de riesgo y características clínicas asociadas con enfermedades cardíacas. El conjunto de datos comprende variables como la edad, el sexo, el tipo de dolor en el pecho, la presión arterial en reposo, el nivel de colesterol, la presencia de azúcar en sangre en ayunas, resultados electrocardiográficos, frecuencia cardíaca máxima alcanzada, presencia de angina inducida por ejercicio, depresión del ST durante el ejercicio, y pendiente del segmento ST durante el ejercicio, entre otros.

El objetivo principal fue utilizar técnicas de análisis de datos y modelado predictivo para comprender mejor la relación entre estas variables y la probabilidad de tener una cardiopatía. Se formularon preguntas e hipótesis específicas para guiar la exploración y el análisis de los datos. El estudio se llevó a cabo en un entorno de Google Colab, aprovechando las herramientas de Python y la biblioteca de aprendizaje automático sklearn.

Los resultados revelaron asociaciones significativas entre ciertas variables y la ocurrencia de cardiopatías, lo que proporcionó una comprensión más profunda de los factores de riesgo involucrados. Se compararon tres modelos de clasificación para poder evaluar el más efectivo en determinar cuándo una persona tiene una cardiopatía. Los modelos más efectivos fueron el de regresión logística y el de vecinos cercanos.

Este estudio contribuye al campo de la salud cardiovascular al proporcionar información valiosa sobre la predicción y la prevención de cardiopatías, lo que podría ayudar a mejorar la atención médica preventiva y reducir la incidencia de esta enfermedad.

Debido al tema tan delicado que se aborda, se priorizó que el modelo detecte la mayor cantidad de casos de pacientes con cardiopatías, por sobre los casos de falsos positivos.

### Hipótesis de interés
1. La edad avanzada está positivamente relacionada con un mayor riesgo de enfermedad cardíaca.
2. Los hombres tienen una mayor incidencia de enfermedad cardíaca en comparación con las mujeres.
3. El tipo de dolor en el pecho específico se asocia con una mayor probabilidad de enfermedad cardíaca.

### Objetivos

Es fundamental destacar que la falta de detección de cardiopatía en un paciente con dicha condición constituye una situación potencialmente trágica que debe ser evitada a toda costa. Es de suma importancia evitar tales escenarios mediante una evaluación exhaustiva y la indicación apropiada de cardiopatía, incluso en ausencia de síntomas claros. La detección temprana y el tratamiento adecuado son cruciales para prevenir complicaciones graves y mejorar el pronóstico de los pacientes con enfermedades cardíacas.
Esta premisa se aplica en todo el trabajo de investigación tanto para la limpieza de los datos como para la elección del modelo.

## Configuración de Entorno

In [80]:
# Librerías utilizadas
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display
import os

# Librería sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [81]:
# Importación de Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/MyDrive/")
%cd "/content/gdrive/MyDrive/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


In [82]:
%run '/content/gdrive/MyDrive/proyecto final/entorno_para_proyecto_final.ipynb'
# Carga del archivo
df_heart_disease = pd.read_csv('/content/gdrive/MyDrive/proyecto final/heart.csv')

## Análisis Exploratorio de Datos

### Variables incorporadas

* **Edad:** medida en años.
* **Sexo:** M: masculino, F: femenino.
* **Dolor_Pecho:** tipo de dolor en el pecho.
  * TA (Typical Angina): angina típica,
  * ATA (Atypical Angina): angina atípica,
  * NAP (Non-Anginal Pain): dolor no anginoso,
  * ASY (Asymptomatic): Asintomático
* **Presión_Arterial_Reposo:** medido en mm Hg.
* **Colesterol:** medido en mm/dl.
* **Azúcar_Sangre_Ayunas:** azúcar en sangre en ayunas. Tomamos la convención siguiente:
  * 1: si azúcar en sangre $>$ 120 mg/dl
  * 0: si azúcar en sangre $\leq$ 120 mg/dl
* **Electrocardiograma_Reposo:** resultados del electrocardiograma en reposo.
  * **Normal:** normal.
  * **ST:** anomalía de la onda ST-T (inversiones de la onda T y/o elevación o depresión del ST > 0,05 mV).
  * **LVH:** mostrando hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes.
* **Frecuencia_Cardíaca_Máxima:** frecuencia cardíaca máxima alcanzada. Valores entre 60 y 202.
* **Angina:** angina inducida por el ejercicio.
  * Y: Yes (sí)
  * N: No
* **Depresión_ST:** mide la depresión del segmento ST en el electrocardiograma (ECG) durante el ejercicio.
* **Pendiente_del_ST:** la pendiente del segmento ST del ejercicio máximo.
  * Up: pendiente positiva.
  * Flat: plana.
  * Down: pendiente negativa.
* **Cardiopatía:** clase de salida.
  * 0: normal.
  * 1: enfermedad cardíaca.

### Validación de datos

In [83]:
# Renombramos las columnas
columns = {
    'Age': 'Edad',
    'Sex': 'Sexo',
    'ChestPainType': 'Dolor_Pecho',
    'RestingBP': 'Presión_Arterial_Reposo',
    'Cholesterol': 'Colesterol',
    'FastingBS': 'Azúcar_Sangre_Ayunas',
    'RestingECG': 'Electrocardiograma_Reposo',
    'MaxHR': 'Frecuencia_Cardíaca_Máxima',
    'ExerciseAngina': 'Angina',
    'Oldpeak': 'Depresión_ST',
    'ST_Slope': 'Pendiente_del_ST',
    'HeartDisease': 'Cardiopatía'
}
df_heart_disease = df_heart_disease.rename(columns=columns)

In [84]:
# Convertimos Azúcar_Sangre_Ayunas y Enfermedad_Cardíaca a variables categóricas
columns = df_heart_disease.columns.tolist()
df_heart_disease['Azúcar_Sangre_Ayunas'] = df_heart_disease['Azúcar_Sangre_Ayunas'].astype('object')
df_heart_disease['Cardiopatía'] = df_heart_disease['Cardiopatía'].astype('object')

In [85]:
# Obtenemos la cantidad de filas y columnas
rows, columns = df_heart_disease.shape
# Obtenemos las columnas categóricas
categorical_data = df_heart_disease.select_dtypes(include=['object'])
categorical_columns = categorical_data.columns.tolist()
# Obtenemos las columnas numéricas
numerical_data = df_heart_disease.select_dtypes(include=['int', 'float'])
numerical_columns = numerical_data.columns.tolist()
# Calculamos la cantidad de valores nulos en total
null_values = calculate_null_values(df_heart_disease)

In [86]:
# Dashboard - Descripción General
table_data = {'Cantidad de variables: ': columns, 'Cantidad de registros: ': rows, 'Valores faltantes (nulos): ': null_values}
# Contamos cantidad de elementos de cada tipo de dato
data_type_count = df_heart_disease.dtypes.value_counts()
df_data_type_count = pd.DataFrame({'type': data_type_count.index, 'conteo': data_type_count.values})
df_data_type_count['type'] = df_data_type_count['type'].astype(str)

# Render
fig = px.pie(df_data_type_count, values='conteo', names='type', hole=.4)
fig.update_layout(width=500, showlegend=True)
fig_html = create_figure(fig)

container = create_flex(
    create_division(
        create_card(create_h(f"""Variables categóricas ({len(categorical_columns)})""") + create_list(categorical_columns)) +
        create_card(create_h(f"""Variables numéricas ({len(numerical_columns)})""") + create_list(numerical_columns))
    ) +
    create_division(create_card(create_formatted_list(table_data, bold=True)) + create_card(fig_html))
, stretch=False)

main_title = create_h('Descripción General', big=True)

render(main_title + create_background(container))

Output hidden; open in https://colab.research.google.com to view.

In [87]:
# Valores cero no permitidos
RestingBP_0 = (df_heart_disease['Presión_Arterial_Reposo'] == 0).sum()
zero_cholesterol = df_heart_disease['Colesterol'] == 0
zero_total_cholesterol = (zero_cholesterol).sum()
heart_disease_and_zero_cholesterol = ((zero_cholesterol) & (df_heart_disease['Cardiopatía'])).sum()
zero_cholesterol_percentage = calculate_percentage(zero_total_cholesterol, rows, 2)

In [88]:
# Dashboard - Imputación de variables

data = ["La presión arterial en reposo y el colesterol no pueden valer cero en un ser humano.",
        f"""Hay {RestingBP_0} caso de presión arterial 0 y {zero_total_cholesterol} casos de colesterol 0.
        De estos últimos, {heart_disease_and_zero_cholesterol} tienen cardiopatías.""",
        f"""Los casos de colesterol 0 representan el {zero_cholesterol_percentage}%
        de los datos. Es una cantidad muy importante del total. Por estos motivos
        concluimos que no deberíamos borrar los casos de valores 0.""",
        """Tanto para la presión arterial como para el colesterol, se procede a
        hacer la imputación por la mediana ya que no es afectada por los valores atípicos.""" ]

# Render
main_title = create_h('Imputación de variables', big=True)
fig = px.pie(df_data_type_count, values='conteo', names='type', hole=.4)
fig.update_layout(width=500, showlegend=True)
fig_html = create_figure(fig)
image = create_image(src='https://i.ibb.co/P1NMxSY/corazon-02.jpg')
container = create_flex(
    create_division(create_card(create_list(data, color='black')) +
                    create_card(create_describe(df_heart_disease.describe().round(1).T))) +
    create_division(image), stretch=False, center=True
)
render(main_title + create_background(container, color='red2'))

,count,mean,std,min,25%,50%,75%,max
Edad,918.0,53.5,9.4,28.0,47.0,54.0,60.0,77.0
Presión_Arterial_Reposo,918.0,132.4,18.5,0.0,120.0,130.0,140.0,200.0
Colesterol,918.0,198.8,109.4,0.0,173.2,223.0,267.0,603.0
Frecuencia_Cardíaca_Máxima,918.0,136.8,25.5,60.0,120.0,138.0,156.0,202.0
Depresión_ST,918.0,0.9,1.1,-2.6,0.0,0.6,1.5,6.2


In [89]:
# Imputamos los valores para el colesterol y la presión arterial en reposo.

# Calculamos la media de los valores distintos de cero en la columna 'Presión_Arterial_Reposo' y 'Colesterol'
median_blood_pressure = df_heart_disease[df_heart_disease['Presión_Arterial_Reposo'] != 0]['Presión_Arterial_Reposo'].median()
median_cholesterol = df_heart_disease[df_heart_disease['Colesterol'] != 0]['Colesterol'].median()

# Reemplazamos los valores cero por la mediana
df_heart_disease.loc[df_heart_disease['Presión_Arterial_Reposo'] == 0, 'Presión_Arterial_Reposo'] = median_blood_pressure
df_heart_disease.loc[df_heart_disease['Colesterol'] == 0, 'Colesterol'] = median_cholesterol

In [90]:
# Dashboard - Descripción de las variables

data_num = [f"""La edad promedio de la muestra es de {int(df_heart_disease['Edad'].mean())} años.""",
        f"""La presión arterial promedio es de {df_heart_disease['Presión_Arterial_Reposo'].mean().round(1)} mmHg y se encuentra dentro del promedio normal.""",
        f"""El colesterol promedio es {df_heart_disease['Colesterol'].mean().round(1)} mm/dl, lo cual está por encima del límite saludable pero podríamos tener presencia de outliers ya que el valor máximo es {df_heart_disease['Colesterol'].max()} mm/dl.""",
        f"""Tanto la frecuencia cardíaca máxima como la depresión del ST durante el ejercicio tienen promedios normales. El valor máximo para la depresión del ST durante el ejercicio, {df_heart_disease['Depresión_ST'].max()}, es bastante alto y podría corresponder a una persona con cardiopatía.""",
        ]
percentage_of_men = calculate_percentage((df_heart_disease['Sexo'] == 'M').sum(), rows, 2)
angina_percentage = calculate_percentage((df_heart_disease['Angina'] == 'Y').sum(), rows, 1)
percentage_of_heart_disease = calculate_percentage((df_heart_disease['Cardiopatía'] == 1).sum(), rows, 1)
percentage_of_asymptomatic = calculate_percentage((df_heart_disease['Dolor_Pecho'] == 'ASY').sum(), rows, 1)
data_cat = [f"""En la muestra tenemos mayor cantidad de hombres ({percentage_of_men}%).""",
        f"""{(df_heart_disease['Dolor_Pecho'] == 'ASY').sum()} pacientes no tienen dolor de pecho ({percentage_of_asymptomatic}%).""",
        f"""La mayoría de los pacientes tiene menos de 120 mg/dl de azúcar en sangre en ayunas.""",
        f"""El resultado más frecuente del electrocardiograma en reposo es "normal".""",
        f"""El {angina_percentage}% de los pacientes tuvo angina inducida por el ejercicio, mientras que el {100 - angina_percentage}% no tuvo.""",
        f"""De los 3 valores para la pendiente del ST durante el ejercicio, el valor más frecuente es "Flat" ({(df_heart_disease['Pendiente_del_ST'] == 'Flat').sum()}).""",
        f"""{percentage_of_heart_disease}% de los pacientes tiene algún tipo de enfermedad cardíaca.""",
        ]

# Render
main_title = create_h("Descripción de las variables", big=True)
container = create_flex(
    create_division(create_card(create_h("Variables numéricas") + create_describe(df_heart_disease.describe().round(1).T) + create_list(data_num))) +
    create_division(create_card(create_h("Variables categóricas") + create_describe(df_heart_disease.describe(include='object').T) + create_list(data_cat)))
)

render(main_title + create_background(container, color='red2'))

## Visualización exploratoria

### Variables categóricas

In [92]:
fig = make_subplots(
    rows=4, cols=2,
    column_widths = [0.5, 0.5],
    row_heights = [0.25, 0.25, 0.25, 0.25],
    specs = [
        [{}, {}],
        [{}, {}],
        [{}, {}],
        [{}, None],
    ]
)

for i, col in enumerate(categorical_columns):
  current_row = i // 2 + 1
  current_column = i % 2 + 1

  for cardiopatia in df_heart_disease['Cardiopatía'].unique():
    hist_data = df_heart_disease[df_heart_disease['Cardiopatía'] == cardiopatia][col]
    fig.add_trace(
        go.Histogram(x=hist_data, hovertemplate='<i>Valor</i>: %{x}<br><b>Cantidad: %{y}</b><extra></extra>'), row=current_row, col=current_column,
        )
    fig.update_xaxes(title_text=col, row=current_row, col=current_column)
    fig.update_yaxes(title_text='Cantidad', row=current_row, col=current_column)

fig.update_layout(
    width = 1200,
    height = 1000,
    template = heart_disease_theme,
    showlegend= False
)

fig.show()

In [93]:
# Cálculos para el insight
people_heart_diseaste = df_heart_disease['Cardiopatía'] == 1
men_heart_disease_percentage = calculate_percentage(((df_heart_disease['Sexo'] == 'M') & (people_heart_diseaste)).sum(), (df_heart_disease['Sexo'] == 'M').sum(), 1)
women_heart_disease_percentage = calculate_percentage(((df_heart_disease['Sexo'] == 'F') & (people_heart_diseaste)).sum(), (df_heart_disease['Sexo'] == 'F').sum(), 1)
asy_percentage = calculate_percentage(((df_heart_disease['Dolor_Pecho'] == 'ASY') & (people_heart_diseaste)).sum(), people_heart_diseaste.sum(), 1)
flat_percentage = calculate_percentage(((df_heart_disease['Pendiente_del_ST'] == 'Flat') & (people_heart_diseaste)).sum(), (df_heart_disease['Pendiente_del_ST'] == 'Flat').sum(), 1)
down_percentage = calculate_percentage(((df_heart_disease['Pendiente_del_ST'] == 'Down') & (people_heart_diseaste)).sum(), (df_heart_disease['Pendiente_del_ST'] == 'Down').sum(), 1)

In [94]:
# Insights sobre variables categóricas
data = [f"""En el gráfico del sexo comprobamos nuestra hipótesis sobre que hay más hombres con cardiopatías que mujeres.
        Pero también encontramos que hay más hombres con cardiopatías ({men_heart_disease_percentage}%) que sanos.
        A su vez, sucede lo contrario para las mujeres (el {women_heart_disease_percentage}% tiene cardiopatías).
        """,
        f"""El {asy_percentage}% de las personas con cardiopatía son asintomáticos. No parece haber tanta relación entre
        los dolores en el pecho y la presencia de cardiopatías, siendo la angina atípica (ATA) la menos relacionada. No
        parece validar nuestra hipótesis sobre la conexión entre el tipo de dolor en el pecho y las cardiopatías, pero
        valiéndonos de otras variables, ser asintomático es un alto factor de riesgo.
        """,
        """La presencia de azúcar en sangre en ayunas indica un gran riesgo para cardiopatías. Lo mismo sucede si hay
        presencia de angina durante el ejercicio
        """,
        f"""El {flat_percentage}% de los pacientes cuya pendiente del ST es 'Flat' tiene una enfermedad cardíaca. A su vez,
        el {down_percentage}% de los que obtuvieron 'Down' también tiene cardiopatía. Esta variable parece ser muy importante mostrando la
        alta probabilidad de tener cardiopatías si nuestro valor es 'Flat' o 'Down'.
        """,
        ]
render(
    create_h("Insights en variables categóricas",big = True) +
    create_background(create_card(create_list(data)))
)

### Variables numéricas

In [95]:
fig = make_subplots(
    rows=4, cols=2,
    column_widths = [0.5, 0.5],
    row_heights = [0.25, 0.25, 0.25, 0.25],
    specs = [
        [{}, {}],
        [{}, {}],
        [{}, {}],
        [{}, None],
    ]
)

for i, col in enumerate(numerical_columns):
  current_row = i // 2 + 1
  current_column = i % 2 + 1

  for cardiopatia in df_heart_disease['Cardiopatía'].unique():
    hist_data = df_heart_disease[df_heart_disease['Cardiopatía'] == cardiopatia][col]
    fig.add_trace(
        go.Histogram(x=hist_data, hovertemplate='%{x}<br><b>Cantidad: %{y}</b><extra></extra>'), row=current_row, col=current_column
        )
    fig.update_xaxes(title_text=col, row=current_row, col=current_column)
    fig.update_yaxes(title_text='Cantidad', row=current_row, col=current_column)

fig.update_layout(
    bargap = 0.2,
    width = 1200,
    height = 1000,
    template = heart_disease_theme,
    showlegend= False
)

fig.show()

In [96]:
# Insights sobre variables numéricas
data = ["""Podemos comprobar nuestra hipótesis sobre la relación entre la edad y el riesgo de tener cardiopatías. A mayor edad,
            efectivamente vemos que el porcentaje de personas con cardiopatías es mayor que las personas sanas. El punto de inflexión
            parece ser entre los 54 y 55 años.
        """,
        f"""Una frecuencia cardíaca máxima alcanzada durante el ejercicio menores a 144 podría indicar cardiopatía.
        """,
        """Depresión del ST en valores por fuera del rango [-0.2, 0.7] han dado mayor porcentaje de personas con cardiopatías que sanos.
        """
        ]
render(
    create_h("Insights en variables numéricas",big = True) +
    create_background(create_card(create_list(data)))
)

### Relación entre variables numéricas y la cardiopatía

In [97]:
heartDisease_dict = {
    0: 'No',
    1: 'Sí'
}

fig = make_subplots(
    rows=2, cols=3,
    column_widths = [0.33, 0.33, 0.33],
    row_heights = [0.50, 0.50],
    specs = [
        [{}, {}, {}],
        [{}, {}, None],
    ]
)

for i, col in enumerate(numerical_columns):
  current_row = i // 3 + 1 # Puede crecer todo lo necesario
  current_column = i % 3 + 1 # La columna es quien limita, por eso 2.

  for cardiopatia in df_heart_disease['Cardiopatía'].unique():
    hist_data = df_heart_disease[df_heart_disease['Cardiopatía'] == cardiopatia][col]
    fig.add_trace(
        go.Box(y=hist_data, name=heartDisease_dict[cardiopatia]), row=current_row, col=current_column
        )
    fig.update_xaxes(title_text='Cardiopatía', row=current_row, col=current_column)
    fig.update_yaxes(title_text=col, row=current_row, col=current_column)

fig.update_layout(
    bargap = 0.2,
    width = 1200,
    height = 1000,
    template = heart_disease_theme,
    showlegend= False
)
fig.show()

In [98]:
# Insights sobre relación entre cardiopatías y variables numéricas

data = ["""Las 3 variables que nos habían llamado la atención en los histogramas
        ('Edad', 'Frecuencia_Cardíaca_Máxima' y 'Pendiente_del_ST')
        parecen verificar nuevamente una relación con las cardiopatías en los boxplots
        que vemos arriba y son variables a tener en cuenta para este análisis.
        """,
        f"""A mayor edad, a menor frecuencia cardíaca máxima alcanzada o valores extremos
        para la depresión del ST, podríamos observar un mayor riesgo de cardiopatía.
        """,
        f"""Además, hay poca presencia de outliers para 'Edad' y
        'Frecuencia_Cardíaca_Máxima', esto los hace muy buenos candidatos para
        estudiar su relación con las cardiopatías.
        """,
        ]
render(
    create_h("Cardiopatía y variables numéricas",big = True) +
    create_background(create_card(create_list(data)))
)

### Relación entre variables categóricas y la cardiopatía

In [99]:
def percentages_by_sex(df, characteristic):
  total_percentages = {}
  types = df_heart_disease[characteristic].unique().tolist()
  types_of_sex = df_heart_disease['Sexo'].unique().tolist()
  people_with_heart_disease = df_heart_disease['Cardiopatía'] == 1
  for sex in types_of_sex:
    total_by_sex = (people_with_heart_disease & (df_heart_disease['Sexo'] == sex))
    percentages_by_chest_pain = {}

    for value in types:
      quantity_characteristic = total_by_sex & (df_heart_disease[characteristic] == value)
      percentages_by_chest_pain[value] = calculate_percentage(quantity_characteristic.sum(), total_by_sex.sum(), 1)

    total_percentages[sex] = percentages_by_chest_pain

  return total_percentages

In [100]:
list_categorical = [
    'Dolor_Pecho',
    'Angina',
    'Azúcar_Sangre_Ayunas',
    'Pendiente_del_ST'
]

layout = ''

fig = make_subplots(
    rows=2, cols=2,
    column_widths = [0.5, 0.5],
    row_heights = [0.50, 0.50],
    specs = [
        [{}, {}],
        [{}, {}],
    ]
)
for i, col in enumerate(list_categorical):
  fig = go.Figure(go.Parcats(
    dimensions=[
        {'label': col, 'values': df_heart_disease[col]},
        {'label': 'Sexo', 'values': df_heart_disease['Sexo']},
        {'label': 'Cardiopatía', 'values': df_heart_disease['Cardiopatía']}
    ],
    line={'color': df_heart_disease['Cardiopatía'], 'colorscale': [[0, BLUE1], [1, RED1]]},
    arrangement='freeform'))
  fig.update_layout(title=f'Relación entre Cardiopatía, Sexo y {col}')
  layout += create_card(create_flex(
    create_division(create_figure(fig), width='75') +
    create_division(create_p(f"Distribución de {col}, por sexo y con cardiopatías.", bold=True, color="black") + create_describe(pd.DataFrame(percentages_by_sex(df_heart_disease, col)).astype('str') + '%'), width='25')
  , center=True, stretch=False))

render(create_background(layout, color='red2'))

Output hidden; open in https://colab.research.google.com to view.

In [101]:
# Insights sobre relación entre cardiopatías y variables numéricas
data = [
    """En todos los gráficos de categorías paralelas se puede comprobar que la
    proporción de hombres con cardiopatía es mayor que los hombres sanos, y a la
    inversa para las mujeres.""",
    f"""En el gráfico del tipo de dolor de pecho vemos que tener ATA podría ser
    un indicio de que no tenemos cardiopatía. En cuanto a las posibles diferencias
    entre sexo, las mujeres con cardiopatía y ATA son el doble que en hombres. El
    caso contrario sucede para TA.""",
    f"""En el gráfico de Angina, vemos que tener Angina es un fuerte indicador de
    cardiopatías. Acá también podemos ver una diferencia entre sexos.""",
    f"""Tener azúcar en sangre en ayunas también parece ser un buen indicador de
    cardiopatías. No se presentan grandes diferencias entre distinto sexo.""",
    f"""Los valores de Flat y Down podrían indicar cardiopatías. No se aprecian
    diferencias significativas entre sexo.""",
    f"""Con estos gráficos y tablas podemos comprobar que la variable sexo también
    puede ser importante para predecir cardiopatías ya que hay diferencias
    sustanciales en algunas características.""",
]

render(
    create_background(create_card(create_list(data)))
)


## Correlaciones

In [102]:
# Convertimos a numérica la columna 'Cardiopatía'
df_heart_disease['Cardiopatía'] = df_heart_disease['Cardiopatía'].astype('int')
numerical_columns = df_heart_disease.select_dtypes(include=['int','float']).columns.tolist()
categorical_columns = df_heart_disease.select_dtypes(include=['object']).columns.tolist()

In [103]:
# Dashboard - Correlación entre variables numéricas
correlations = df_heart_disease[numerical_columns].corr()
correlations_sort = correlations['Cardiopatía'].sort_values(ascending=False)[1:].round(3)
corrPos = [
    "Depresión_ST_durante_ejercicio",
    "Edad"
    ]
corrNeg = [
    "Frecuencia_Cardíaca_Máxima_Alcanzada"
]


# Render
main_title = create_h("Correlación entre variables numéricas",big=True)
conclusion = create_p("En el gráfico de calor podemos ver en forma visual que las variables correlacionadas con la enfermedad cardíaca son las que habíamos podido analizar en los gráficos de cajas.", color='black')
conclusion += create_p('Correlación positiva:',color='black', bold = True ) + create_list(corrPos, color='black')
conclusion += create_p('Correlación negativa:',color='black', bold = True ) + create_list(corrNeg, color='black')
fig = px.imshow(correlations, labels=dict(color="Correlación"), x = correlations.index, y = correlations.columns, color_continuous_scale='reds')
fig.update_xaxes(side="top")
fig.update_layout(
    coloraxis_colorbar=dict(
        x=0.8,  # Posición horizontal de la barra de colores (0 a 1)
    ),
)
container = create_flex(
        create_division(create_card(create_h("Correlación respecto de Cardiopatía") + create_formatted_list(correlations_sort, bold=True)) + create_card(conclusion)) +
        create_division(create_card(create_figure(fig, center = True))), stretch = False
    )
render(main_title + create_background(container, color = 'red2'))

Output hidden; open in https://colab.research.google.com to view.

Las variables que mayor relación parecen tener son la pendiente del ST y la depresión ST durante el ejercicio, la edad, la frecuencia cardíaca máxima alcanzada durante el ejercicio, la angina inducida por el ejercicio, el sexo y el tipo de dolor en el pecho.

El mejor modelo de machine learning que podemos utilizar para predecir cardiopatías es el de aprendizaje supervisado para poder categorizar si un paciente presenta o no alguna enfermedad cardiopática.

## Modelo

Vamos a comparar tres modelos y analizar cuál es el más óptimo para predecir cardiopatías. Utilizaremos:
* KNN MODEL
* DECISION TREE MODEL
* LOGISTIC MODEL

In [104]:
# Información categórica
categorical_data = df_heart_disease.select_dtypes(include=['object'])
categorical_columns = categorical_data.columns.tolist()
# Transformamos las variables categóricas a valores numéricos para el modelo
df_heart_disease = encode_categorical_features(df_heart_disease, categorical_columns)
df_heart_disease

,Edad,Sexo,Dolor_Pecho,Presión_Arterial_Reposo,Colesterol,Azúcar_Sangre_Ayunas,Electrocardiograma_Reposo,Frecuencia_Cardíaca_Máxima,Angina,Depresión_ST,Pendiente_del_ST,Cardiopatía
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,1,3,110,264,0,1,132,0,1.2,1,1
914,68,1,0,144,193,1,1,141,0,3.4,1,1
915,57,1,0,130,131,0,1,115,1,1.2,1,1
916,57,0,1,130,236,0,0,174,0,0.0,1,1


In [105]:
# Separo X, y
X = df_heart_disease.drop('Cardiopatía', axis=1)
y = df_heart_disease['Cardiopatía']

# Divido en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalo los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Análisis para obtener el K óptimo para KNN.

In [106]:
scores = []
for k in range(1,21):
    # Creamos el modelo KNN
    knn = KNeighborsClassifier(n_neighbors=k)

    # Lo entrenamos
    knn.fit(X_train_scaled, y_train)

    # Hacemos las predicciones
    y_pred = knn.predict(X_test_scaled)
    # Calcular el puntaje de precisión y almacenarlo
    scores.append({'k':k, 'accuracy': accuracy_score(y_test, y_pred)})

# pd.DataFrame(classification_report(y_test, y_pred, output_dict=True))
# Encontrar el valor de k con el mejor puntaje de precisión
best_k = max(scores, key=lambda x: x['accuracy'])['k']

scores = pd.DataFrame(scores)
scores.set_index(scores['k'], inplace=True)
scores.drop(columns = ['k'], inplace=True)

# # Crear el gráfico de barras horizontal
fig = go.Figure(go.Bar(
    x=scores['accuracy'],
    y=scores.index,
    orientation='h',
    marker=dict(color = scores.index.map(lambda x: RED1 if x == best_k else BLUE1)),
    width=0.9,
    hovertemplate='Accuracy: %{x}<br><b>K: %{y}</b><extra></extra>'
))

# # Actualizar diseño del gráfico
fig.update_layout(
    xaxis_title='Accuracy',
    yaxis_title='Valor de k',
    yaxis=dict(autorange='reversed'),
    height=600,
    width=1000
)
container = create_flex(
    create_division(create_card(
        create_p("Se probaron valores de k desde el 1 hasta el 20 inclusive") +
        create_p(f"El mejor valor de k es {best_k}",bold=True,color='red')
    ) ) +
    create_division(create_card(create_figure(fig)))

)

container = create_card(
    create_h(f"Para valores k de 1 a 20 (inclusive) la mayor exactitud la obtenemos con un k igual a {best_k}.") +
    create_figure(fig, center = True)
)
render(create_h('Precisión del Modelo para diferentes valores de k', big=True) + create_background(container, color='red2'))

Output hidden; open in https://colab.research.google.com to view.

## Aplicación de cada modelo para comparar

In [109]:
# KNN MODEL
# =========
# Inicializo y entreno el modelo de KNN
modelKNN = KNeighborsClassifier(n_neighbors=best_k)
modelKNN.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_predictionsKNN = modelKNN.predict(X_test_scaled)
confusionKNN = pd.DataFrame(confusion_matrix(y_test, y_predictionsKNN), columns=["0", "1"], index=["0", "1"])

# DECISION TREE MODEL
# ===================
# Inicializo y entreno el modelo de DecisionTree
modelDecisionTree = DecisionTreeClassifier()
modelDecisionTree.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_predictionsDecisionTree = modelDecisionTree.predict(X_test)
confusionDecisionTree = pd.DataFrame(confusion_matrix(y_test, y_predictionsDecisionTree), columns=["0", "1"], index=["0", "1"])

# LOGISTIC MODEL
# ==============
# Inicializo y entreno el modelo de regresión logística
modelLogistic = LogisticRegression()
modelLogistic.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_predictionsLogistic = modelLogistic.predict(X_test_scaled)
confusionLogistic = pd.DataFrame(confusion_matrix(y_test, y_predictionsLogistic), columns=["0", "1"], index=["0", "1"])

In [110]:
models = [
    {
        'title': 'KNN',
        'model': modelKNN,
        'predictions': y_predictionsKNN,
        'plot': confusionKNN
    },
    {
        'title': 'Árbol de decisiones',
        'model': modelDecisionTree,
        'predictions': y_predictionsDecisionTree,
        'plot': confusionDecisionTree
    },
    {
        'title': 'Regresión Logística',
        'model': modelLogistic,
        'predictions': y_predictionsLogistic,
        'plot': confusionLogistic
    },
]

# Render
for model in models:
  reporte= create_report(pd.DataFrame(classification_report(y_test, model['predictions'], output_dict=True)), model['plot'])

  score_container = create_flex(
      create_division(
          create_card(create_h("Sin cardiopatía") + create_formatted_list((reporte['score_values']['0']).to_dict(),unit = '%', bold = True, separate = True))
      ) +
      create_division(
          create_card(create_h("Con cardiopatía") + create_formatted_list((reporte['score_values']['1']).to_dict(),unit = '%', bold = True, separate = True))
      )
      ,stretch = False
  )

  figure = reporte['confusion_matrix']

  figure.update_layout(
      coloraxis_colorbar=dict(
          x=0.8,
      ),
  )

  container = create_flex(
      create_division(
          create_card(create_h(f"Accuracy: {reporte['acc']}")) +
          # create_card(create_h(f"Accuracy: {reporte['acc']}", big = True)) +
          create_card(create_h("Sin cardiopatía") + create_formatted_list((reporte['score_values']['0']).to_dict(),unit = '%', bold = True, separate = True))+
          create_card(create_h("Con cardiopatía") + create_formatted_list((reporte['score_values']['1']).to_dict(),unit = '%', bold = True, separate = True))


      ) +
      create_division(
          # score_container +
          create_card(create_figure(figure)) +
          create_card(create_describe(reporte['report']))
      ), stretch = False
  )
  render(create_h(f"Reporte de modelo - {model['title']}",big = True) + create_background(container))

El F1-score para KNN y regresión logística es alto, esto nos indica que el modelo tiene un buen equilibrio entre precisión y recall, lo que sugiere que es efectivo en la clasificación de las clases objetivo. Por otro lado, Árbol de decisiones tiene un F1-score similar a los anteriores pero es un poco más bajo, es decir, es un poco más ineficiente para predecir las clases. El F1-score que más valoramos es el de cardiopatías que es incluso mejor que cuando no hay cardiopatías. Esto es porque necesitamos que cuando el modelo indique que hay cardiopatías, efectivamente sea una persona con dicha condición. A su vez, tenemos que priorizar el recall con cardiopatías ya que queremos que todas las personas con dicha condición puedan ser diagnosticadas. El equilibrio entre ambas (medido por el F1-score) indica que los mejores modelos son KNN y regresión logística.

# Conclusiones

In [111]:
# Dashboard - Conclusiones
coef_df = pd.DataFrame(data = modelLogistic.coef_[0], index = X.columns)
coef_df = coef_df.rename(columns={0: 'Coeficiente'})
coef_df['Coeficiente'] = abs(coef_df['Coeficiente'])
coef_df = coef_df.sort_values(by='Coeficiente', ascending=False)

data_models = [
    """De los modelos evaluados, los mejores fueron KNN y regresión logística,
    siendo ambos exactamente iguales en las mediciones.""",
    """Valoramos que el recall cuando hay cardiopatía sea lo más alto posible,
    sin perder rendimiento para las demás métricas. Como el f1-score es muy
    bueno, existe un buen equilibrio entre la precisión y el recall y por lo tanto,
    ambos modelos pueden ser adoptados para predecir cardiopatías en pacientes.""",
]

data_coef = [
        """En la tabla de arriba, podemos ver los coeficientes obtenidos a partir del
    modelo de regresión logística. Se aprecia que hemos podido determinar como
    importantes varias variables en el análisis exploratorio que coinciden con
    los coeficientes del modelo.""",
        """Las variables Pendiente_del_ST, Angina, Sexo, Dolor_Pecho,
        Azúcar_Sangre_Ayunas y Depresión_ST son las variables más importantes para
        el modelo. """,
]

data = [
        """Si bien las diferencias de edades no se validan en el modelo (no muestra
        una relación tan directa entre la edad avanzada y las cardiopatías) sí pudimos
        apreciar en el análisis exploratorio que a mayor edad habían más personas con
        cardiopatías que sanas.""",
        """El sexo sí es una variable importante, además, nuestra hipótesis queda confirmada
        en que hay más hombres que padecen cardiopatías que mujeres. En estudios
        posteriores se puede hacer hincapié en las posibles diferencias entre
        sexos para poder predecir con mayor exactitud o hacer modelos específicos
        para cada sexo.""",
        """La variable Dolor_Pecho también está fuertemente relacionada pero hay
        que tener en cuenta que realmente la mayoría de los casos son
        asintomáticos, es decir, la mayoría de las personas con cardiopatías no
        tienen realmente dolores en el pecho, por lo que no confirma nuestra
        hipótesis que se basaba en que algún tipo de dolor en particular fuera
        más proclive que otro a indicarnos cardiopatías.""",
        """Dolor_Pecho nos indica algo muy importante sobre estas enfermedades.
        La mayoría de las personas que presentan cardiopatías no tienen ningún
        tipo de dolor. Esto es muy importante para concientizar a la población
        sobre la importancia de hacerse estudios periódicos y no autodiagnosticarse."""
]

# Render
container = create_flex(
    create_division(create_card(create_h("Modelos") + create_list(data_models, color='black'))) +
    create_division(create_card(create_describe(coef_df)))
)
container += create_card(create_h("Coeficientes") + create_list(data_coef,bullets=False))
container += create_card(create_h("Nuestras hipótesis") + create_list(data,bullets=False))
render(create_h("Conclusiones", big=True) + create_background(container, color='red2'))

,Coeficiente
Pendiente_del_ST,1.011026
Angina,0.581269
Sexo,0.578568
Dolor_Pecho,0.552688
Azúcar_Sangre_Ayunas,0.482526
Depresión_ST,0.446149
Frecuencia_Cardíaca_Máxima,0.297807
Edad,0.145266
Electrocardiograma_Reposo,0.083721
Presión_Arterial_Reposo,0.055362
